<a href="https://colab.research.google.com/github/alinaalborova/russian_idioms_processing/blob/main/IdiomSim_ruBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Idiom Paraphrase Experiments


Based on [Idiom Paraphrases: Seventh Heaven vs Cloud Nine](https://www.aclweb.org/anthology/W15-2709v2.pdf) by Maria Pershina Yifan He Ralph Grishman.


## Import Libraries

In [ ]:
!pip install transformers
!pip install tensor2tensor

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm_notebook

## Load Dataset

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
dataset_dir = '/content/drive/MyDrive/ВКР/fedorov_clean.csv'
data = pd.read_csv(dataset_dir )
data.head()

,Unnamed: 0,Idiom,Definition
0,0,а вот на тебе!,Выражение удивления разочарования и т п
1,1,и вот на тебе!,Выражение удивления разочарования и т п
2,2,а был ли мальчик?,Было ли что либо в действительности на самом д...
3,3,а вот,Напротив наоборот
4,4,а вот и нет,Не выходит нельзя сделать то что намечается пр...


In [ ]:
data.shape

(14326, 3)

In [ ]:
data.drop('Unnamed: 0', axis=1, inplace=True)
data.drop(0, inplace=True)
data

,Idiom,Definition
1,и вот на тебе!,Выражение удивления разочарования и т п
2,а был ли мальчик?,Было ли что либо в действительности на самом д...
3,а вот,Напротив наоборот
4,а вот и нет,Не выходит нельзя сделать то что намечается пр...
5,вот и нет,Не выходит нельзя сделать то что намечается пр...
...,...,...
14321,ямская изба,В России XIII XVIII вв почтовая станция где пр...
14322,ясная голова,О том кто способен логично чётко мыслить
14323,яснее ясного,Совершенно понятно очевидно
14324,ясное дело,Ясно же конечно безусловно


In [ ]:
data.dropna(inplace=True)
data.shape

(13940, 2)

In [ ]:
data['Definition'].value_counts()

Умирать                                                              26
Умереть                                                              14
Божба клятвенное заверение в чём либо                                10
Очень много                                                           9
Совершенно забываться                                                 7
                                                                     ..
Самые лучшие талантливые дееспособные полезные для общества люди      1
По внешнему виду                                                      1
Фамильяр Лучше сесть чем стоять Обращение с приглашением сесть        1
В состоянии гнева будучи сердитым рассерженным делать что либо        1
О чём либо выходящем из пределов обычного привычного                  1
Name: Definition, Length: 10928, dtype: int64

## Annoy

In [ ]:
!pip install annoy
from tqdm import tqdm_notebook
import networkx as nx       
from annoy import AnnoyIndex

     |████████████████████████████████| 655kB 27.7MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=391612 sha256=785dc9be1c6b16e93eae0652a341677e036bef0bdece6fb53dc48c59c88cd031
  Stored in directory: /root/.cache/pip/wheels/3a/c5/59/cce7e67b52c8e987389e53f917b6bb2a9d904a03246fadcb1e
Successfully built annoy


In [ ]:
f = 768
Index_lex = AnnoyIndex(f, 'angular')  # Length of item vector to be indexed
Index_sem = AnnoyIndex(f, 'angular')  # Length of item vector to be indexed

## Get Embeddings

Definitions --> BERT --> Definitions embeddings

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel, TFAutoModel
  
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")

BERT_model = AutoModel.from_pretrained('DeepPavlov/rubert-base-cased')

### Embed

In [ ]:
inputs = tokenizer('пример предложения. ', return_tensors="pt")
outputs=BERT_model(**inputs)

In [ ]:
def extract_full_embeddings(lhs_all, padded_size=60):
  extracted = []
  for lhs in lhs_all:
    sentence_vecs = lhs[0][0]# lhs for all sentences
    print('Sentence vecs', len(sentence_vecs))
    extracted.append(sentence_vecs) 
  print('Extracted', len(extracted))
  return extracted

def get_bert_embeddings_single_sentence(sentence):
  inputs = tokenizer(sentence, return_tensors="pt")
  outputs=BERT_model(**inputs)
  return outputs.last_hidden_state
  
def get_bert_embeddings(sentences, padded_size=60, vec_size=768):
  embeddings_all = []
  #for sentence in tqdm_notebook(sentences):
  for sentence in tqdm_notebook(sentences):
    lhs = get_bert_embeddings_single_sentence(sentence)
    vecs = lhs[0]
    sentence_padded = np.empty(shape=(padded_size, ), dtype=object)
    for id, token in enumerate(sentence_padded):
      try:
        sentence_padded[id] = vecs[id].detach().numpy()
      except:
        sentence_padded[id] = np.zeros(vec_size)
    embeddings_all.append(sentence_padded)
  return embeddings_all

## Similarity Tricks

#### _Auxiliary_. Build TF-IDF

In [ ]:
tk = tf.keras.preprocessing.text.Tokenizer()
tk.fit_on_texts(data['Definition'])

tk_output = tk.sequences_to_matrix(tk.texts_to_sequences(data['Definition']), mode='tfidf')

In [ ]:
print(len(data['Definition']))
len(tk_output) == len(data['Definition'])

13940


True

In [ ]:
len(tk_output[0]) ## vocab size

20606

In [ ]:
from tqdm import tqdm_notebook

In [ ]:
def align_tfidf(tk_output):
  weights_all = []
  for doc in tqdm_notebook(tk_output):
    weights = [float(0), ] # reserved for cls
    for w in doc:
      if w != float(0):
        weights.append(w)
    while len(weights) < MAX_LEN:
      weights.append(float(0))
    weights_all.append(weights)
  return weights_all

In [ ]:
MAX_LEN = 60
tfidf_values = align_tfidf(tk_output)
len(tfidf_values)

13940

### Lexical Similarity

Compute cosine similarity between vectors Vd1 and Vd2, representing idiom descriptions d1 and d2 and weight each word in these vectors by its tf-idf score:  
$lexSim(d1, d2) = cosine(\vec{v}_{d1}, \vec{v}_{d2})$

where $\vec{v}$ d is a $\left |V \right |$-dimensional vector with $V$
being the vocabulary of all definition words.

1. Get Vd1 and Vd2, representing idiom descriptions d1 and d2 and weight each word in these vectors by its tf-idf score

In [ ]:
def get_all_vd_values(tokens, tfidf_values):
  """
  Multplies each token vector in an idiom by its tf-idf value.
  """
  vd_values_total = []
  for id, full_token in enumerate(tokens): # for each sentence's tokens
    vd_values = []
    for i, el in enumerate(full_token):
      token_emb_by_tfidf = tf.math.multiply(el, tfidf_values[id][i])
      vd_values.append(token_emb_by_tfidf)
    #vd_values_total.append(tf.math.reduce_mean(vd_values, axis=0))
    vd_values_total.append(np.mean(vd_values, axis=0))

  return vd_values_total

In [ ]:
def get_vd_values(definition):
  '''
  idiom_tokens: list of strings
  '''
  idiom_id = list(data.Definition).index(definition)
  try:
    idiom_tokens = full_tokens[idiom_id]
  except:
    print(idiom_id)
  idiom_tfidf_values = tfidf_values[idiom_id]
  vd_values = []
  for i, el in enumerate(idiom_tokens):
    token_emb_by_tfidf = tf.math.multiply(el, idiom_tfidf_values[i])
    vd_values.append(token_emb_by_tfidf)  
  return tf.math.reduce_mean(vd_values, axis=0)

In [ ]:
def calculate_lex_sim(i1_id, i2_id):
  return 1 - id_vd.get_distance(i1_id, i2_id)

### Semantical Similarity

To capture the overall meaning of the definitions d we combine word embeddings
for all words in d using two combination schemes:
* Averaged sum:  
$\vec{averaged}_{d} = \frac{1}{\left |d  \right |}\sum_{word \in d} \vec{emb}(word)$
* Weighted sum:   
$\vec{weighted}_{d} = \frac{1}{\sum_{word \in d}tfidf_{word}}\sum_{word \in d}tfidf_{word} \cdot  \vec{emb}(word)$   
<br><br>

Then semantic similarity is measured as $semSim(d1,d2)= cosine(comb_{d1}, comb_{d2})$,  
where
$\vec{comb}_{d}$ is a 100-dimensional vector combined from word embeddings
$\vec{emb}(word)$ or words in description $d$ using either averaged or weighted combination
schemes. 


#### Averaged Sum

$\vec{averaged}_{d} = \frac{1}{\left |d  \right |}\sum_{word \in d} \vec{emb}(word)$

get average of all words in d

In [ ]:
def get_avg_sum(definition_tokens):
  return np.sum(definition_tokens, axis=0)

#### semSim

Semantic similarity is measured as $semSim(d1,d2)= cosine(comb_{d1}, comb_{d2})$,  
where
$\vec{comb}_{d}$ is a 100-dimensional vector combined from word embeddings
$\vec{emb}(word)$ or words in description $d$ using either averaged or weighted combination
schemes. 

2. Calculate semantic similarity

In [ ]:
def calculate_sem_sim_avg(i1_id, i2_id):
  return 1 - id_vd.get_distance(i1_id, i2_id)

## Operating through batches

In [ ]:
data.shape

(13940, 2)

In [ ]:
size = [i for i in range(13940)]
ranges = size[::500]
ranges

[0,
 500,
 1000,
 1500,
 2000,
 2500,
 3000,
 3500,
 4000,
 4500,
 5000,
 5500,
 6000,
 6500,
 7000,
 7500,
 8000,
 8500,
 9000,
 9500,
 10000,
 10500,
 11000,
 11500,
 12000,
 12500,
 13000,
 13500]

In [ ]:
for r in ranges:
  full_tokens = get_bert_embeddings(data['Definition'][r:r+500])
  vd_values = get_all_vd_values(full_tokens, tfidf_values)
  for index, vector in enumerate(vd_values):
    Index_lex.add_item(index+r, vector)
  del vd_values
  avg_sums = []
  for definition in full_tokens:
    avg_sums.append(np.sum(definition, axis=0))
  for index, vector in enumerate(avg_sums):
    Index_sem.add_item(index+r, vector)
  del avg_sums

In [ ]:
def build_index(index):

  index.build(100)

  return index

In [ ]:
id_vd = build_index(Index_lex)

In [ ]:
id_avg = build_index(Index_sem)

### IdiomSim

* lexSim captures the actual lexical overlap
* semSim can better capture the closeness in semantic meaning

To find an optimal balance between the two we consider their weighted sum:  
$IdiomSim(d1, d2) =  (1 - \alpha) \cdot  lexSim(d1, d2) + \alpha \cdot semSim(d1, d2)$
and decide on an $\alpha$ by optimizing for a maximal
F-score on a development dataset.

In [ ]:
#alpha = 0

## Using Average scheme for semantic similarity measurements
def calculate_idiom_sim_avg(idiom1, idiom2, alpha=0.5):
  i1_id = list(data.Idiom).index(idiom1)
  i2_id = list(data.Idiom).index(idiom2)
  #print('Lexical Similarity ', calculate_lex_sim(i1_id, i2_id))
  #print('Semantical Similarity ', calculate_sem_sim_avg(i1_id, i2_id))
  similarity_rate = (1 - alpha) * calculate_lex_sim(i1_id, i2_id) + alpha * calculate_sem_sim_avg(i1_id, i2_id)
  #print('Definition #1: {}\nDefinition #2: {}\nSimilarity rate:'.format(d1, d2, similarity_rate))
  return similarity_rate

# Test

In [ ]:
data_test = data.sample(500, random_state=42)
data_test

,Idiom,Definition
11342,разводить тары-бары (растабары),Заниматься пустословием попусту болтать
10145,показать кукиш,То же что показывать дулю i кому А ты чай дума...
13580,устоять в слове,Выполнить обещанное
13289,точить балясы (лясы),Пустословить заниматься пустой болтовнёй Перво...
7802,не иметь духа (духу),Не отважиться не решиться что либо сказать сде...
...,...,...
10917,прищемлять хвост,Строго ограничивать кого либо в его действиях ...
6674,музейная редкость,Что либо давно уже вышедшее из употребления из...
10239,положение хуже губернаторского,Очень неприятное крайне затруднительное положе...
7938,не помнить себя,От волнения от каких либо переживаний не отдав...


In [ ]:
def find_similar_idioms_(idiom1, idioms, method='average', top_k=5):
  similarities = {}
  calculate_sim = calculate_idiom_sim_avg

  for id, idiom in enumerate(tqdm_notebook(idioms)):
    #print(id, idiom)
    if id < 13440:
      similarities[id] = calculate_sim(idiom1, idiom)
  sorted_dict = {}
  sorted_values = sorted(similarities.values())

  for i in sorted_values:
    for k in similarities.keys():
      if similarities[k] == i:
        sorted_dict[k] = similarities[k]
        break

  sorted_list = list(sorted_dict.items())
  sorted_list.reverse()
  candidates = []
  for i, id in enumerate(sorted_list[1 :top_k + 1]):
    candidates.append([idioms[id[0]], list(data.Definition)[id[0]]])
    
  return candidates

In [ ]:
test_size = [i for i in range(501)]
test_ranges = test_size[::10]
test_ranges

[0,
 10,
 20,
 30,
 40,
 50,
 60,
 70,
 80,
 90,
 100,
 110,
 120,
 130,
 140,
 150,
 160,
 170,
 180,
 190,
 200,
 210,
 220,
 230,
 240,
 250,
 260,
 270,
 280,
 290,
 300,
 310,
 320,
 330,
 340,
 350,
 360,
 370,
 380,
 390,
 400,
 410,
 420,
 430,
 440,
 450,
 460,
 470,
 480,
 490,
 500]

In [ ]:
rows_all = []
for r in test_ranges[10:21]:
  print(r, '/500_____________________\n')
  rows = []
  for i, idiom in enumerate(data_test.Idiom.values[r:r+10]):
    row = [[idiom, list(data.Definition)[list(data.Idiom).index(idiom)], ]]
    print(idiom, list(data.Definition)[list(data.Idiom).index(idiom)], list(data.Idiom).index(idiom))
    candidates = find_similar_idioms_(idiom, list(data.Idiom))
    for candidate in candidates:
      row.append(candidate)
    rows.append(row)
  rows_all += rows

predictions = pd.DataFrame(rows_all)

100 /500_____________________

преградить дорогу Мешать препятствовать кому либо в чём либо  10355



отсеивать плевелы от зёрен Разделять разграничивать плохое и хорошее полезное и бесполезное  8956



на словах Устно в устной форме Только в разговоре а не на деле  6925



купить ценой Получить право возможность поступать каким либо образом за счёт каких либо потерь  6014



старый свет Части света Европа Азия Африка в отличие от открытой позже Америки Нового Света  12562


...


с горизонта пропадать, пропасть, исчезнуть, исчезать b Из какой либо общественной среды из круга друзей знакомых соратников и т п  11396



до основания Совершенно полностью разрушить разорить потрясти и т п  3429



по назначению В назначенное место передать послать  9353



оста(ва)ться ни при чём Оставаться без того на что рассчитывал надеялся кто либо  8625



не пройдёт (прошло) даром Не исчезает бесследно не оказывается бесполезным безрезультатным что либо Не остаётся безнаказанным кто либо  7744


In [ ]:
predictions.to_csv('/content/drive/MyDrive/IdiomSim/predictions_ruBERT2.csv', encoding='utf-8-sig')